# Bratu Equation

This is the implementation for Bratu Equation in 2D using FEniCSx.
The boundary-value problem is described by:
$$
\begin{aligned}
-\Delta u(x) = \lambda e^{u(x)} &\quad x\in \Omega \\
u(x) = 0 &\quad x\in \partial \Omega
\end{aligned}
$$

## Variational formulation

To calculate the variational formulation, we multiply the equation by a test function $v$ and integrate over $\Omega$:
$$\int_\Omega (-\Delta u)v dx = \int_\Omega \lambda e^{u(x)} v dx$$

Using integration by parts to the LHS:
$$\int_\Omega \nabla u \cdot \nabla v dx - \int_{\partial\Omega} \frac{\partial u}{\partial n} v ds
    = \int_\Omega \lambda e^{u(x)} v dx$$
Since the integral term on the boundary is 0, the equation becomes
$$\int_\Omega \nabla u \cdot \nabla v dx = \int_\Omega \lambda e^{u(x)} v dx$$
leading to
$$
F(u,v) := \int_\Omega \nabla u \cdot \nabla v dx - \int_\Omega \lambda e^{u(x)} v dx
$$

## Problem definition

- Domain: $\Omega = [0,1] \times [0,1]$
- Dirichlet BCs: $u = 0$ on $\partial\Omega$
- Forcing fn: $\lambda = 1$

In [25]:
# Import Libraries
from mpi4py import MPI
from dolfinx import mesh, fem, plot, io, log
import numpy as np
import ufl
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc
import pyvista
from pathlib import Path

In [ ]:
# Create mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 50, 50, cell_type=mesh.CellType.triangle)

# Create FunctionSpace
V = fem.functionspace(domain, ("Lagrange", 1))

# Apply Dirichlet BCs
# Identify the facets on the left and right boundaries
facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, lambda x: np.full(x.shape[1], True, dtype=bool))
# Locate DoFs on the facets
dofs_boundary = fem.locate_dofs_topological(V, domain.topology.dim - 1, facets)
# Create Dirichlet BCs
bc = fem.dirichletbc(PETSc.ScalarType(0.0), dofs_boundary, V)

# Define variational problem
# Define trial and test functions
u = fem.Function(V)
v = ufl.TestFunction(V)
w = ufl.TrialFunction(V)
x = ufl.SpatialCoordinate(domain)
lambda_ = fem.Constant(domain, PETSc.ScalarType(1.0))
# Define residual
F = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx - lambda_ * ufl.exp(u) * v * ufl.dx
# Define Jacobian
J = ufl.inner(ufl.grad(w), ufl.grad(v)) * ufl.dx - lambda_ * ufl.exp(u) * w * v * ufl.dx

# Create NonlinearProblem
problem = NonlinearProblem(F, u, [bc], J)

# Create Newton Solver
solver = NewtonSolver(domain.comm, problem)
# Set solver options
solver.atol = 1e-8
solver.rtol = 1e-7
solver.max_it = 50

# Solve nonlinear problem
log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(u)

# Define residual
residual = fem.Form(F)
jacobian = fem.Form(J)
if converged:
    print(f"Newton solver converged in {n} iterations.")
else:
    print("Newton solver did NOT converge!")

# Post-process
# Save solution to file
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "out_bratu"
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w", io.XDMFFile.Encoding.ASCII) as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(u)

[2025-02-23 16:31:13.263] [info] Extract basic topology: 15000->15000
[2025-02-23 16:31:13.263] [info] Build local dual graph
[2025-02-23 16:31:13.263] [info] Build local part of mesh dual graph (mixed)
[2025-02-23 16:31:13.264] [info] GPS pseudo-diameter:(198) 4901-98
[2025-02-23 16:31:13.264] [info] Create topology (single cell type)
[2025-02-23 16:31:13.264] [info] Create topology (generalised)
[2025-02-23 16:31:13.264] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2025-02-23 16:31:13.264] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2025-02-23 16:31:13.265] [info] Compute ghost indices
[2025-02-23 16:31:13.265] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2025-02-23 16:31:13.265] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2025-02-23 16:31:13.265] [info] Computing communication graph edges (using NBX algo